In [ ]:
%matplotlib inline
import splat
import splat.database as spd
import splat.plot as splot
import splat.empirical as spem
import pandas
import numpy
from scipy.interpolate import interp1d
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import astropy.constants as constants
import astropy.units as u
from astropy.io import ascii, fits            # for reading in spreadsheet
bfold = '/Users/adam/papers/toi2406/'
sxdfile = bfold+'spex-sxd_TOI212957629_20201203.fits'


In [ ]:
def readsxd(file,output='1dspec',name='',**kwargs):
    funit = u.erg/u.s/u.cm/u.cm/u.Angstrom
    wunit = u.micron
    xrngs = [[1.95,2.47],[1.43,1.81],[1.1,1.5],[0.92,1.2],[0.83,1.02],[0.735,0.93],[0.7,0.8]]
    with fits.open(file, **kwargs) as hdulist:
        header = hdulist[0].header
        meta = {'header': header}
        data = hdulist[0].data
    spec = []
    orders = header['ORDERS'].split(',')
    for i in range(len(data[:,0,0])):
        sp = splat.Spectrum(wave=data[i,0,:]*wunit,flux=data[i,1,:]*funit,noise=data[i,2,:]*funit,header=header,instrument='SPEX-SXD',name='{} order {}'.format(name,orders[i]))
        sp.trim(xrngs[i])
        spec.append(sp)
        
    if output=='multispec': return spec
    elif output=='1dspec':
        spc = spec[0]
        for s in spec[1:]: spc = splat.stitch(spc,s,scale=False)
        spc.name=name
        return spc
        
    else: return data



In [ ]:
xrng = [0.83,1.02]
s = readsxd(sxdfile,name='TOI-2406',output='1dspec')
s.plot(yrange=[-1.e-16,5.e-15])

In [ ]:
for r in ['rojas','terrien','mann13','mann14','newton']:
    spem.metallicity(s,ref=r,verbose=True)

In [ ]:
# classify smoothed spectrum
s.reset()
s.smooth(30)
s.name='TOI-2406'
splat.classifyByStandard(s,plot=True,method='kirkpatrick',telluric=True)

In [ ]:
splat.searchLibrary(back=True)

In [4]:
import splat.empirical as spem
spem.typeToColor('M6','PANSTARRS_R-2MASS_K',set='best')

(5.949999999999999, 0.05)

In [ ]:
a = [4,5,6]
a.index(5)